In [1]:
import numpy as np
import pandas as pd

from data.dataset import StockDataset
from data.scaler import HybridScaler
from data.split import StratifiedTimeSeriesSplit
from model.arima import grid_search
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate

import warnings
warnings.filterwarnings('ignore')

# Data

In [2]:
dataset = StockDataset('^GSPC')
df = dataset.get_hist(start_date='1950-01-01', end_date='2021-10-23', time_interval='daily')

In [3]:
x = pd.concat(
    [df, 
     dataset.lookback_agg(lookback_len=30), 
     dataset.lookback_agg(lookback_len=60), 
     dataset.lookback_agg(lookback_len=120)],
    axis=1)

y = dataset.get_change_forecast_label(forecast_len=30, is_up=False, method='past_all')

In [4]:
TRAIN_START = '1951-01-01'
TEST_START = '2018-01-01'

train_x = x[TRAIN_START:TEST_START]
train_y = y[TRAIN_START:TEST_START]
test_x = x[TEST_START:]
test_y = y[TEST_START:]

split = TimeSeriesSplit(n_splits=10, test_size=120)
# split = StratifiedTimeSeriesSplit(n_splits=10, test_size=120, min_positive_ratio=0.25)

# LogisticRegression

In [6]:
pipeline = Pipeline(steps=[
    ('scaler', HybridScaler()), 
    ('model', LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga', class_weight='balanced'))
])

In [7]:
results = cross_validate(pipeline, train_x, train_y, cv=split, scoring='accuracy')

In [8]:
results['test_score'].mean()

0.8033333333333333

In [9]:
pipeline.fit(train_x, train_y)
pred_y = pipeline.predict(test_x)

confusion_matrix(test_y, pred_y)

array([[237, 400],
       [135, 188]])

# RandomForecast

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
pipeline = Pipeline(steps=[
    ('scaler', HybridScaler()), 
    ('model', RandomForestClassifier(n_estimators=100, random_state=1))
])

In [12]:
results = cross_validate(pipeline, train_x, train_y, cv=split, scoring='accuracy')

In [13]:
results['test_score'].mean()

0.8608333333333332

In [14]:
pipeline.fit(train_x, train_y)
pred_y = pipeline.predict(test_x)

confusion_matrix(test_y, pred_y)

array([[400, 237],
       [114, 209]])